In [66]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
plt.rcParams["figure.figsize"]=8,8
import math
import numpy as np
import pandas as pd
from scipy import stats
import scipy
import scipy.stats as stat
from scipy.interpolate import make_interp_spline, BSpline
from lmfit.models import Model
import cv2 as cv2
# !pip install scikit-image
import skvideo.io
import skimage.color
import os
import random as rng

In [ ]:
# THINGS TO ADD
# - Centriod identifier and record x, y
# - Adjust frame rate calculation since video is sped up
# - Calculate velocity (over large distance or instantaneous)
# - 

In [89]:
nu_o = 1.824e-5 # N-s/m^2 Viscosity of air, uncorrected for radius
vy_o = 8.77e-5 # m/s Droplet downward velocity with no field
vy_E = 1 # m/s Droplet downward velocity with field E
g = 9.8 # m/s^2 Acceleration of gravity
rho = 8.86e2 # kg/m^3 Density of oil droplet
b = 8.20e-3 #N/m Viscosity correction factor
p = 1.013e+5 #N/m^2 pressure

# 5/11/2022
# 2.131 ± 0.001 MΩ 
# 0.5 ± 0.05 kV
# 7.53 ± 0.001 mm
# 22°C 

In [94]:
a = ((9*nu_o*vy_o/(2*g*rho))+(b/(2*p)**2)**2)**0.5 - (b/(2*p))
V = 500
d = 7.53
print(a)

8.700442354138149e-07


In [98]:
q = 4/3*np.pi*a**3*rho*g*d/V*(vy_E-vy_o)/vy_o
print(q/1.602176634e-19,"vs. lit",1.602176634e-19)

25671385.23923775 vs. lit 1.602176634e-19


In [86]:
# IMAGE BINARY MASK PROCESSING
# https://docs.opencv.org/4.x/dd/dd7/tutorial_morph_lines_detection.html
# Load the image
videosrc = cv2.VideoCapture("test1_compressed.mp4")
success = 1
count = 0
frames = 285
img_array = []
ct = CentroidTracker()
while success and (count < frames):
    success, src = videosrc.read()
    if type(src) == type(None):
        break
    height, width, layers = src.shape
    size = (width,height)
    # Transform source image to gray if it is not already
    if len(src.shape) != 2:
        gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    else:
        gray = src
    gray = thresh_callback(150,255,gray,ct)
    cv2.imwrite(f'frame_{count}.png',gray)
    count+=1
    img_array.append(f'frame_{count}.png')
    # [smooth]

output = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)-1):
    img = cv2.imread(img_array[i])
#     print(f"Adding file {img_array[i]} ")
    output.write(img)
    os.remove(img_array[i])
output.release()

In [85]:
# !rm frame_*.png
# BOUNDING BOXES
def thresh_callback(lower_thresh,upper_thresh,src_gray, ct):
    ret, src_gray = cv2.threshold(src_gray, lower_thresh, upper_thresh,cv2.THRESH_BINARY)
    canny_output = cv2.Canny(src_gray, lower_thresh, upper_thresh)
    contours, _ = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # SETUP ARRAYS
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    centers = [None]*len(contours)
    radius = [None]*len(contours)
    # FIND EACH BOUNDING BOX AND CIRCLE
    for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
        centers[i], radius[i] = cv2.minEnclosingCircle(contours_poly[i])
    drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
    # DRAW EACH BOUNDING BOX AND CIRCLE
    for i in range(len(contours)):
        color = (0, 255, 0) # green
#         cv2.drawContours(drawing, contours_poly, i, color)
        x,y,w,h = boundRect[i]
        ct.upadate(boundRect[i])
        cv2.rectangle(drawing, (int(x), int(y)), (int(x+w), int(y+h)), color, 1)
        cv2.putText(drawing, f'{i}', (int(x+w)+10,int(y+h)),0,0.3,(0,255,0))
#         cv2.circle(drawing, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), color, 2)
    for (objectID, centroid) in objects.items():
		# draw both the ID of the object and the centroid of the
		# object on the output frame
		text = "ID {}".format(objectID)
		cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
		cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
    return drawing

In [87]:
# CENTROID TRACKER
# https://pyimagesearch.com/2018/07/23/simple-object-tracking-with-opencv/#download-the-code
from scipy.spatial import distance as dist
from collections import OrderedDict
class CentroidTracker():
	def __init__(self, maxDisappeared=50):
		# initialize the next unique object ID along with two ordered
		# dictionaries used to keep track of mapping a given object
		# ID to its centroid and number of consecutive frames it has
		# been marked as "disappeared", respectively
		self.nextObjectID = 0
		self.objects = OrderedDict()
		self.disappeared = OrderedDict()
		# store the number of maximum consecutive frames a given
		# object is allowed to be marked as "disappeared" until we
		# need to deregister the object from tracking
		self.maxDisappeared = maxDisappeared
	def register(self, centroid):
		# when registering an object we use the next available object
		# ID to store the centroid
		self.objects[self.nextObjectID] = centroid
		self.disappeared[self.nextObjectID] = 0
		self.nextObjectID += 1
	def deregister(self, objectID):
		# to deregister an object ID we delete the object ID from
		# both of our respective dictionaries
		del self.objects[objectID]
		del self.disappeared[objectID]
	def update(self, rects):
		# check to see if the list of input bounding box rectangles
		# is empty
		if len(rects) == 0:
			# loop over any existing tracked objects and mark them
			# as disappeared
			for objectID in list(self.disappeared.keys()):
				self.disappeared[objectID] += 1
				# if we have reached a maximum number of consecutive
				# frames where a given object has been marked as
				# missing, deregister it
				if self.disappeared[objectID] > self.maxDisappeared:
					self.deregister(objectID)
			# return early as there are no centroids or tracking info
			# to update
			return self.objects
		# initialize an array of input centroids for the current frame
		inputCentroids = np.zeros((len(rects), 2), dtype="int")
		# loop over the bounding box rectangles
		for (i, (startX, startY, endX, endY)) in enumerate(rects):
			# use the bounding box coordinates to derive the centroid
			cX = int((startX + endX) / 2.0)
			cY = int((startY + endY) / 2.0)
			inputCentroids[i] = (cX, cY)
		# if we are currently not tracking any objects take the input
		# centroids and register each of them
		if len(self.objects) == 0:
			for i in range(0, len(inputCentroids)):
				self.register(inputCentroids[i])
		# otherwise, are are currently tracking objects so we need to
		# try to match the input centroids to existing object
		# centroids
		else:
			# grab the set of object IDs and corresponding centroids
			objectIDs = list(self.objects.keys())
			objectCentroids = list(self.objects.values())
			# compute the distance between each pair of object
			# centroids and input centroids, respectively -- our
			# goal will be to match an input centroid to an existing
			# object centroid
			D = dist.cdist(np.array(objectCentroids), inputCentroids)
			# in order to perform this matching we must (1) find the
			# smallest value in each row and then (2) sort the row
			# indexes based on their minimum values so that the row
			# with the smallest value is at the *front* of the index
			# list
			rows = D.min(axis=1).argsort()
			# next, we perform a similar process on the columns by
			# finding the smallest value in each column and then
			# sorting using the previously computed row index list
			cols = D.argmin(axis=1)[rows]
			# in order to determine if we need to update, register,
			# or deregister an object we need to keep track of which
			# of the rows and column indexes we have already examined
			usedRows = set()
			usedCols = set()
			# loop over the combination of the (row, column) index
			# tuples
			for (row, col) in zip(rows, cols):
				# if we have already examined either the row or
				# column value before, ignore it
				# val
				if row in usedRows or col in usedCols:
					continue
				# otherwise, grab the object ID for the current row,
				# set its new centroid, and reset the disappeared
				# counter
				objectID = objectIDs[row]
				self.objects[objectID] = inputCentroids[col]
				self.disappeared[objectID] = 0
				# indicate that we have examined each of the row and
				# column indexes, respectively
				usedRows.add(row)
				usedCols.add(col)
			# compute both the row and column index we have NOT yet
			# examined
			unusedRows = set(range(0, D.shape[0])).difference(usedRows)
			unusedCols = set(range(0, D.shape[1])).difference(usedCols)
			# in the event that the number of object centroids is
			# equal or greater than the number of input centroids
			# we need to check and see if some of these objects have
			# potentially disappeared
			if D.shape[0] >= D.shape[1]:
				# loop over the unused row indexes
				for row in unusedRows:
					# grab the object ID for the corresponding row
					# index and increment the disappeared counter
					objectID = objectIDs[row]
					self.disappeared[objectID] += 1
					# check to see if the number of consecutive
					# frames the object has been marked "disappeared"
					# for warrants deregistering the object
					if self.disappeared[objectID] > self.maxDisappeared:
						self.deregister(objectID)
			# otherwise, if the number of input centroids is greater
			# than the number of existing object centroids we need to
			# register each new input centroid as a trackable object
			else:
				for col in unusedCols:
					self.register(inputCentroids[col])
		# return the set of trackable objects
		return self.objects

In [ ]:
    ct = CentroidTracker()
    # update our centroid tracker using the computed set of bounding
	# box rectangles
	objects = ct.update(rects)
	# loop over the tracked objects
	for (objectID, centroid) in objects.items():
		# draw both the ID of the object and the centroid of the
		# object on the output frame
		text = "ID {}".format(objectID)
		cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
		cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)